In [ ]:
# Before running this code, please make sure the following steps are completed:
# 1. Upgrade the robot actuator communication firmware version to 0.3.12.25 or above.
# 2. Calibrate the robot joint zero position(usually done by the manufacturer).
# 3. Use the latest docker image for development.

In [ ]:
# fourier_hardware_py is the main library for the hardware control
# grx_sot_py is the library for robot kinematics and dynamics calculation
import fourier_hardware_py
import grx_sot_py
import numpy as np
import time
import math

In [ ]:
# create a configuration for GR2T2 robot
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr2t2v2")

In [ ]:
# create an IMU object and initialize it
# Note: 
# If the imu is not connected to the computer when the docker container starts, the imu's serial port information cannot be obtained correctly in the docker container. 
# In this case, restart the docker container.
ch108 = fourier_hardware_py.ch108IMU()
ch108.initIMU("/dev/ttyUSB0")
# Ater initialization, you can get the imu information from the config object.
imu_names = config.getIMUNames()
first_imu_name = imu_names[0]
print(first_imu_name)
print(config.getIMUType(first_imu_name[0])) # imu type
print(config.getIMUMountLink(first_imu_name)) # imu link in urdf
print(config.getIMUPortName(first_imu_name)) # imu port
for i in range(2):
    print(ch108.getIMUData()) # w x y z wx wy wz ax ay az 
    print(ch108.getLossCount()) # number of lost packets, max 255
    time.sleep(0.1) 

In [ ]:
# set PD control mode for each control group 
config.changeControlGroupMotorMode("head","pd")
config.changeControlGroupMotorMode("left_manipulator","pd")
config.changeControlGroupMotorMode("right_manipulator","pd")
config.changeControlGroupMotorMode("waist","pd")
config.changeControlGroupMotorMode("left_leg","pd")
config.changeControlGroupMotorMode("right_leg","pd")

True

In [ ]:
# create a robot hardware object and build connections.
# Note:
# If the control group cannot connect, the code will keep printing error messages, please check if the robot joint ip can be pinged,
# if not, please try to restart the robot or seek hardware support.
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/gr2t2v2",config)

In [ ]:
# create a robot wrapper for kinematics and dynamics calculation
robot = grx_sot_py.RobotWrapper("/usr/local/config/fourier_hardware/gr2t2v2", grx_sot_py.Flags.ignore_collisions)

In [ ]:
# print robot control group and joint information
group_names = []
group_joints = []
group_joints_pos_order = {}
group_joint_vel_order = {}

for group_name in config.getControlGroupNames():
    if config.getControlGroupMotorMode(group_name)== "ignore":
        continue
    group_names.append(group_name)
    robot_joint_names = hardware.getControlGroupJointNames(group_name)
    group_joints.append(robot_joint_names)
    joint_pos_orders = []
    joint_vel_orders = []
    for robot_joint_name in robot_joint_names:
        joint_pos_orders.append(robot.get_joint_offset(robot_joint_name))
        joint_vel_orders.append(robot.get_joint_v_offset(robot_joint_name))
    group_joints_pos_order[group_name]= joint_pos_orders
    group_joint_vel_order[group_name] = joint_vel_orders

In [8]:
print(config.getRobotName())
print(config.getControlGroupNames())

gr2t2v2
['left_leg', 'right_leg', 'waist', 'head', 'left_manipulator', 'right_manipulator']


In [19]:
hardware.setControlGroupPIDParams("left_leg",[50,50,50,50,20,10],[0,0,0,0,0,0],[0,0,0,0,0,0])
hardware.setControlGroupPIDParams("right_leg",[50,50,50,50,20,10],[0,0,0,0,0,0],[0,0,0,0,0,0])
hardware.setControlGroupPIDParams("waist",[60],[0],[0])
hardware.setControlGroupPIDParams("head",[20,20],[0,0],[0,0])
hardware.setControlGroupPIDParams("right_manipulator",[300,100,50,50,50,50,50],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0])
hardware.setControlGroupPIDParams("left_manipulator",[300,100,50,50,50,50,50],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0])

True

In [ ]:
# Enable the robot joint motors
hardware.enableRobot()

MOTOR: 192.168.137.93, ENABLE SUCCESS! 
MOTOR: 192.168.137.95, ENABLE SUCCESS! 
MOTOR: 192.168.137.70, ENABLE SUCCESS! 
MOTOR: 192.168.137.71, ENABLE SUCCESS! 
MOTOR: 192.168.137.72, ENABLE SUCCESS! 
MOTOR: 192.168.137.73, ENABLE SUCCESS! 
MOTOR: 192.168.137.75, ENABLE SUCCESS! 
MOTOR: 192.168.137.74, ENABLE SUCCESS! 
MOTOR: 192.168.137.10, ENABLE SUCCESS! 
MOTOR: 192.168.137.11, ENABLE SUCCESS! 
MOTOR: 192.168.137.12, ENABLE SUCCESS! 
MOTOR: 192.168.137.13, ENABLE SUCCESS! 
MOTOR: 192.168.137.14, ENABLE SUCCESS! 
MOTOR: 192.168.137.15, ENABLE SUCCESS! 
MOTOR: 192.168.137.16, ENABLE SUCCESS! 
MOTOR: 192.168.137.50, ENABLE SUCCESS! 
MOTOR: 192.168.137.51, ENABLE SUCCESS! 
MOTOR: 192.168.137.52, ENABLE SUCCESS! 
MOTOR: 192.168.137.53, ENABLE SUCCESS! 
MOTOR: 192.168.137.55, ENABLE SUCCESS! 
MOTOR: 192.168.137.54, ENABLE SUCCESS! 
MOTOR: 192.168.137.30, ENABLE SUCCESS! 
MOTOR: 192.168.137.31, ENABLE SUCCESS! 
MOTOR: 192.168.137.32, ENABLE SUCCESS! 
MOTOR: 192.168.137.33, ENABLE SUCCESS! 


True

In [ ]:
# get the joint positions(rad) of the control groups ['waist', 'head', 'left_manipulator', 'right_manipulator', 'left_leg', 'right_leg'] in the current robot state.
lm = hardware.getControlGroupState("left_manipulator")
rm = hardware.getControlGroupState("right_manipulator")
print("left_manipulator:", lm.q)
print("right_manipulator", rm.q)

left_manipulator: [ 0.          0.          0.          0.          0.          0.
  0.         -0.00349039  0.00143703 -0.00019981  0.00528092  0.00020083
 -0.00170787  0.0003646 ]
right_manipulator [ 0.          0.          0.          0.          0.          0.
  0.         -0.00668046 -0.00477701 -0.00377041  0.08985603 -0.0103717
 -0.0046316  -0.01243471]


In [ ]:
# generate the trajectory of the robot's arms from the current position to the zero position
point_init = [-0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
rm_traj = fourier_hardware_py.bridgeTrajectory(rm.q[-7:],point_init)
lm_traj = fourier_hardware_py.bridgeTrajectory(lm.q[-7:],point_init)

In [ ]:
# excute the trajectory genetated above to move the right arm to zero position
import time

for i in range(len(rm_traj[1])):
    start_time = time.perf_counter()
    # get the real robot state
    state = hardware.readRealRobotState()
    # Note: 
    # Due to the error in the observation of joint velocity and joint acceleration, and the delay caused by subsequent matrix multiplication, 
    # the Coriolis force and inertial force are not considered here, only the gravity is considered (set state.qd and state.qdd to zero).
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # calculate the joint compensation torque and use PD feedback to compensate the torque to improve control accuracy
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    # set the control command for the right arm
    hardware.setControlGroupPosCmd("right_manipulator",rm_traj[1][i],np.zeros(7),gravity_torque[group_joint_vel_order["right_manipulator"]])
    state = hardware.readRealRobotState()

    end_time = time.perf_counter()
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [ ]:
# execute the trajectory generated to move the left manipulator to zero position
for i in range(len(lm_traj[1])):
    start_time = time.perf_counter()
    state = hardware.readRealRobotState()
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("left_manipulator",lm_traj[1][i],np.zeros(7),gravity_torque[group_joint_vel_order["left_manipulator"]])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [ ]:
# generate the trajectory of the robot's waist joint from the current position to the zero position
waist_point_init = [-0.0]
waist_state = hardware.getControlGroupState("waist")
print(waist_state.q)
waist_traj = fourier_hardware_py.bridgeTrajectory(waist_state.q[-1:], waist_point_init)

[0.         0.         0.         0.         0.         0.
 0.         0.05787623]


In [ ]:
# exceute the trajectory generated above to move the waist to zero position
for i in range(len(waist_traj[1])):
    start_time = time.perf_counter()
    state = hardware.readRealRobotState()
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("waist",waist_traj[1][i])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [ ]:
# generate the trajectory of the robot's head joint from the current position to the zero position
head_point_init = [0.0, 0.0]
head_state = hardware.getControlGroupState("head")
print(head_state.q)
head_traj = fourier_hardware_py.bridgeTrajectory(head_state.q[-2:], head_point_init)

[ 0.          0.          0.          0.          0.          0.
  0.         -0.07118839  0.08118156]


In [ ]:
# exceute the trajectory generated above to move the head to zero position
for i in range(len(head_traj[1])):
    start_time = time.perf_counter()
    
    state = hardware.readRealRobotState()
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("head",head_traj[1][i])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [ ]:
# generate the trajectory of the robot's left leg from the current position to the zero position
left_leg_point_init = [0.0, 0.0,0.0,0.0, 0.0, 0.0]
left_leg_state = hardware.getControlGroupState("left_leg")
print(left_leg_state.q)
left_leg_traj = fourier_hardware_py.bridgeTrajectory(left_leg_state.q[-6:], left_leg_point_init)

[ 0.          0.          0.          0.          0.          0.
  0.         -0.44437818  0.01144917  0.08791179  0.42361097 -0.29711867
 -0.01354484]


In [ ]:
# excecute the trajectory genetated to move the left leg to zero position
for i in range(len(left_leg_traj[1])):
    start_time = time.perf_counter()
    state = hardware.readRealRobotState()
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("left_leg",left_leg_traj[1][i])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [ ]:
# generate the trajectory of the robot's right leg from the current position to the zero position
right_leg_point_init = [0.0, 0.0,0.0,0.0, 0.0, 0.0]
right_leg_state = hardware.getControlGroupState("right_leg")
print(right_leg_state.q)
right_leg_traj = fourier_hardware_py.bridgeTrajectory(right_leg_state.q[-6:], right_leg_point_init)

[ 0.          0.          0.          0.          0.          0.
  0.         -0.37289606  0.03074731  0.14017145  0.37487368 -0.30617617
  0.01289041]


In [ ]:
# excecute the trajectory genetated to move the right leg to zero position
for i in range(len(right_leg_traj[1])):
    start_time = time.perf_counter()
    state = hardware.readRealRobotState()
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    hardware.setControlGroupPosCmd("right_leg",right_leg_traj[1][i])
    state = hardware.readRealRobotState()
    end_time = time.perf_counter()
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

In [ ]:
# calculate and print the gravity torque
gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
print(gravity_torque)
print(gravity_torque[group_joint_vel_order["left_manipulator"]])

[-3.10000365e-03 -4.84231530e-03  6.07732396e+02 -6.41495937e+00
  1.90972705e+01  1.89363015e-03 -2.37918257e+00  1.22717004e+00
 -4.57144545e-01  4.16662888e+00  5.93708219e-03 -3.91233890e-01
  1.66646919e+01 -1.20043858e+00 -3.56332761e-01  7.76378233e+00
 -1.49506666e-03 -9.35917836e-02  5.07350019e-04  3.98534162e-06
 -4.38111461e-01 -2.90889742e-01  1.36533127e-02 -9.92231193e-04
 -1.17515184e-01 -7.61478786e-04 -4.86104274e-02 -2.11125685e-03
 -2.95321962e-01 -1.96549986e-01  5.99203300e-04 -1.22503087e-01
  2.26971335e-04 -5.00821472e-02 -1.85259033e-03]
[-0.29088974  0.01365331 -0.00099223 -0.11751518 -0.00076148 -0.04861043
 -0.00211126]


In [ ]:
position_desired = np.zeros(17)  # deg
ERROR_CODE = 50 

# set the frequency of the loop
sim_dt = 0.0025
sim_duration = 60.00
sim_steps = int(sim_duration / sim_dt)

for k in range(sim_steps):
    start_time = time.perf_counter()
    # current control
    position_desired[0] = (15 * math.sin(0.0025 * k - math.pi / 2) + 15) # waist_yaw_joint

    position_desired[1] = -(15 * math.sin(0.0025 * k)) # head_yaw_joint
    position_desired[2] = -(15 * math.sin(0.0025 * k)) # head_pitch_joint
    
    position_desired[3] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_pitch_joint
    position_desired[4] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # left_shoulder_roll_joint
    position_desired[5] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_yaw_joint
    position_desired[6] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_elbow_pitch_joint
    position_desired[7] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_wrist_yaw_joint
    position_desired[8] = -(10 * math.sin(0.0025 * k)) # left_wrist_roll_joint
    position_desired[9] = -(10 * math.sin(0.0025 * k)) # left_wrist_pitch_joint
    
    position_desired[10] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_pitch_joint
    position_desired[11] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_roll_joint
    position_desired[12] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_shoulder_yaw_joint
    position_desired[13] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_elbow_pitch_joint
    position_desired[14] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_wrist_yaw_joint
    position_desired[15] = -(10 * math.sin(0.0025 * k)) # right_wrist_roll_joint
    position_desired[16] =  -(10 * math.sin(0.0025 * k)) # right_wrist_pitch_joint

    # convert joint position from degree to radian
    position_desired = np.deg2rad(position_desired)

    # return the number of times the control group has lost connection continuously (the maximum observation value is 50)
    motor_stats = hardware.getGroupsLoss()
    # check the hardware status of the real robot, if any motor is disconnected, the control loop will be interrupted and return an error message
    if ERROR_CODE in motor_stats.values():
        print(f"motor get error.")
        break

    state = hardware.readRealRobotState()
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    gravity_torque = robot.get_joint_compensation_torque(state.q,state.qd,state.qdd)
    
    # execute the trajectory
    hardware.setControlGroupPosCmd("left_manipulator",position_desired[3:10],np.zeros(7),gravity_torque[group_joint_vel_order["left_manipulator"]])
    hardware.setControlGroupPosCmd("right_manipulator",position_desired[10:17],np.zeros(7),gravity_torque[group_joint_vel_order["right_manipulator"]])
    hardware.setControlGroupPosCmd("waist", np.array([position_desired[0]]))
    hardware.setControlGroupPosCmd("head", position_desired[1:3])
    
    end_time = time.perf_counter()
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)


In [14]:
hardware.disableRobot()

MOTOR: 192.168.137.93, DISABLE SUCCESS! 


True

MOTOR: 192.168.137.95, DISABLE SUCCESS! 
MOTOR: 192.168.137.10, DISABLE SUCCESS! 
MOTOR: 192.168.137.11, DISABLE SUCCESS! 
MOTOR: 192.168.137.12, DISABLE SUCCESS! 
MOTOR: 192.168.137.13, DISABLE SUCCESS! 
MOTOR: 192.168.137.14, DISABLE SUCCESS! 
MOTOR: 192.168.137.15, DISABLE SUCCESS! 
MOTOR: 192.168.137.16, DISABLE SUCCESS! 
MOTOR: 192.168.137.30, DISABLE SUCCESS! 
MOTOR: 192.168.137.31, DISABLE SUCCESS! 
MOTOR: 192.168.137.32, DISABLE SUCCESS! 
MOTOR: 192.168.137.33, DISABLE SUCCESS! 
MOTOR: 192.168.137.34, DISABLE SUCCESS! 
MOTOR: 192.168.137.35, DISABLE SUCCESS! 
MOTOR: 192.168.137.36, DISABLE SUCCESS! 
MOTOR: 192.168.137.90, DISABLE SUCCESS! 
[2025-02-28 09:55:32] [12182] [error] head loss message more than 50 times .
[2025-02-28 09:55:32] [12182] [error] left_manipulator loss message more than 50 times .
[2025-02-28 09:55:32] [12182] [error] right_manipulator loss message more than 50 times .
[2025-02-28 09:55:33] [12182] [error] head loss message more than 50 times .
[2025-02-28